# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# Load pretrained VGG16
vgg16 = models.vgg16(pretrained=True)
print(vgg16)

# Modify classifier for CIFAR-10
vgg16.classifier[6] = torch.nn.Linear(4096, 10)

# Freezing backbone for finetuining
for param in vgg16.parameters():
    param.requires_grad = False

for param in vgg16.classifier.parameters():
    param.requires_grad = True

for name, param in vgg16.named_parameters():
    status = "Trainable" if param.requires_grad else "Frozen"
    print(f"{name}: {status}")

# Move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

In [ ]:
# Transformations
transform = transforms.Compose([
    transforms.Resize(224),  # resize to default vgg16 input size
    transforms.ToTensor(),
])

# Loading CIFAR-10
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=1e-4, momentum = 0.9)

In [5]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()  # Set model to training mode

    # Lists to store loss and accuracy values for each epoch
    epoch_losses = []
    epoch_accuracies = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        # Wrap train_loader with tqdm for progress tracking
        with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # Backward pass and optimization
                loss.backward()
                optimizer.step()

                # Statistics
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                # Update the progress bar
                pbar.set_postfix(loss=running_loss / (pbar.n + 1), accuracy=100. * correct / total)
                pbar.update(1)

        # Calculate epoch statistics
        epoch_loss = running_loss / len(train_loader)
        accuracy = 100. * correct / total
        epoch_losses.append(epoch_loss)
        epoch_accuracies.append(accuracy)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')

    # Plot the training loss and accuracy graphs
    fig, ax1 = plt.subplots()

    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color='tab:red')
    ax1.plot(range(1, num_epochs + 1), epoch_losses, color='tab:red', label='Loss')
    ax1.tick_params(axis='y', labelcolor='tab:red')

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    ax2.set_ylabel('Accuracy (%)', color='tab:blue')  # we already handled the x-label with ax1
    ax2.plot(range(1, num_epochs + 1), epoch_accuracies, color='tab:blue', label='Accuracy')
    ax2.tick_params(axis='y', labelcolor='tab:blue')

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.title("Training Loss and Accuracy over Epochs")
    plt.show()

In [ ]:
train_model(vgg16, train_loader, criterion, optimizer, num_epochs=3)
torch.save(vgg16.state_dict(), 'vgg16_t2_cifar10.pth')

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    # List to store individual accuracies for each batch (optional)
    batch_accuracies = []

    with torch.no_grad():
        # Wrap test_loader with tqdm for progress tracking
        with tqdm(total=len(test_loader), desc='Evaluating', unit='batch') as pbar:
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                # Calculate accuracy for the current batch (optional)
                batch_accuracy = 100. * predicted.eq(labels).sum().item() / labels.size(0)
                batch_accuracies.append(batch_accuracy)

                # Update progress bar with the current batch accuracy
                pbar.set_postfix(batch_accuracy=batch_accuracy)
                pbar.update(1)

    accuracy = 100. * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

    # Plotting batch accuracies (optional)
    if batch_accuracies:
        plt.figure(figsize=(10, 5))
        plt.plot(batch_accuracies, label='Batch Accuracy', color='blue')
        plt.xlabel('Batch Number')
        plt.ylabel('Accuracy (%)')
        plt.title('Batch Accuracy during Evaluation')
        plt.axhline(y=accuracy, color='red', linestyle='--', label='Overall Accuracy')
        plt.legend()
        plt.grid()
        plt.show()

In [ ]:
evaluate_model(vgg16, test_loader)

In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F
import math

def show_predictions(model, loader, device, num_samples=50, cols=10):
    # Switch model to evaluation mode
    model.eval()

    # Get a batch of data from the loader
    images, labels = next(iter(loader))
    images, labels = images.to(device), labels.to(device)

    # Get the model's predictions
    with torch.no_grad():
        outputs = model(images)
        _, predicted_labels = torch.max(outputs, 1)

    # Convert tensor to numpy for visualization
    images = images.cpu().numpy()
    labels = labels.cpu().numpy()
    predicted_labels = predicted_labels.cpu().numpy()

    # Plot the images with predicted and true labels
    rows = math.ceil(num_samples / cols)  # Calculate number of rows based on num_samples and cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i in range(num_samples):
        ax = axes[i]
        img = images[i].transpose((1, 2, 0))  # Change format to HWC for plotting
        img = img * 255.0  # Assuming the images are normalized, revert back to original pixel range
        img = img.astype('uint8')  # Convert to uint8 for display

        ax.imshow(img)
        ax.axis('off')
        ax.set_title(f"True: {labels[i]}\nPred: {predicted_labels[i]}")

    # Turn off axes for any extra subplots (if num_samples doesn't fill the grid perfectly)
    for i in range(num_samples, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Show 50 sample images with their predictions in a grid of 10 columns
show_predictions(vgg16, train_loader, device, num_samples=50, cols=10)
